# Кодировка свечей по методу Лиховидова, прогноз Up или Down.

Обучение модели на основе кода свечи Лиховидова.  
Бинарная классификация Up или Down.  
Основной гиперпараметр seed. Подбирается по графикам.  
Вот обновленный код с использованием взвешивания классов в функции потерь (BCEWithLogitsLoss с pos_weight). Я добавил вычисление pos_weight на основе количества классов и передал его в функцию потерь.

In [87]:
import sqlite3
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import random
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

# === Функция для фиксации случайных чисел ===
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_var = 79
set_seed(seed_var)

# === Загрузка данных ===
db_path = Path(r'C:\Users\Alkor\gd\data_quote_db\MIX_futures_day.db')
with sqlite3.connect(db_path) as conn:
    df_fut = pd.read_sql_query(
        """
        SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME 
        FROM Day 
        ORDER BY TRADEDATE
        """,
        conn
    )

# === Функция кодирования свечей ===
def encode_candle(row):
    open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']
    direction = 1 if close > open_ else (0 if close < open_ else 2)
    upper_shadow = high - max(open_, close)
    lower_shadow = min(open_, close) - low
    body = abs(close - open_)
    
    def classify_shadow(shadow, body):
        return 0 if shadow < 0.1 * body else (1 if shadow < 0.5 * body else 2)

    return f"{direction}{classify_shadow(upper_shadow, body)}{classify_shadow(lower_shadow, body)}"

df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# === Преобразование кодов в числа ===
unique_codes = sorted(df_fut['CANDLE_CODE'].unique())
code_to_int = {code: i for i, code in enumerate(unique_codes)}
df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

window_size = 20  
predict_offset = 1  

# === Подготовка данных. Создание последовательностей (датасетов). ===
X, y = [], []
for i in range(len(df_fut) - window_size - predict_offset):
    X.append(df_fut['CANDLE_INT'].iloc[i:i+window_size].values)
    y.append(
        1 if df_fut['CLOSE'].iloc[i+window_size+predict_offset] > 
        df_fut['CLOSE'].iloc[i+window_size] else 0
    )

X, y = np.array(X), np.array(y)

# === Разделение на обучающую и тестовую выборки ===
split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# === Взвешивание классов ===
class_counts = np.bincount(y_train)
# pos_weight = torch.tensor([class_counts[0] / class_counts[1]], dtype=torch.float32)
pos_weight = torch.tensor([class_counts[0] / class_counts[1]]) if class_counts[1] > 0 else torch.tensor([1.0])

# === Создание DataLoader ===
class CandlestickDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# === Инициализация генератора случайных чисел для каждого воркера ===
def seed_worker(worker_id):
    np.random.seed(42 + worker_id)
    random.seed(42 + worker_id)

train_dataset = CandlestickDataset(X_train, y_train)
test_dataset = CandlestickDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, worker_init_fn=seed_worker)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, worker_init_fn=seed_worker)

# === Определение модели ===
class CandleLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(CandleLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  
        return x  # Без sigmoid, так как используем BCEWithLogitsLoss

# === Выбор устройства ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Инициализация модели ===
model = CandleLSTM(
    vocab_size=len(unique_codes), embedding_dim=8, hidden_dim=32, output_dim=1
).to(device)

# === Использование взвешивания классов ===
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(device))
optimizer = optim.Adam(model.parameters(), lr=0.001)

best_accuracy = 0  
epoch_best_accuracy = 0
model_path = "best_model_first_MIX_01.pth"
early_stop_epochs = 200
epochs_no_improve = 0

epochs = 2000
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch).squeeze()
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = torch.sigmoid(model(X_batch).squeeze()).round()
            correct += (y_pred == y_batch).sum().item()
            total += y_batch.size(0)

    accuracy = correct / total
    print(
        f"Epoch {epoch+1}/{epochs}, "
        f"Loss: {total_loss/len(train_loader):.4f}, "
        f"Test Accuracy: {accuracy:.2%}, "
        f"Best accuracy: {best_accuracy:.2%}, "
        f"Epoch best accuracy: {epoch_best_accuracy}, "
        f"seed: {seed_var}"
    )

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        epochs_no_improve = 0
        epoch_best_accuracy = epoch + 1
        torch.save(model.state_dict(), model_path)
        print(f"✅ Model saved with accuracy: {best_accuracy:.2%}")
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_epochs:
        print(f"🛑 Early stopping at epoch {epoch + 1}")
        break

print("\n🔹 Loading best model for final evaluation...")
model.load_state_dict(torch.load(model_path))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred = torch.sigmoid(model(X_batch).squeeze()).round()
        correct += (y_pred == y_batch).sum().item()
        total += y_batch.size(0)

final_accuracy = correct / total
print(f"🏆 Final Test Accuracy: {final_accuracy:.2%}")


Epoch 1/2000, Loss: 0.6734, Test Accuracy: 44.58%, Best accuracy: 0.00%, Epoch best accuracy: 0, seed: 79
✅ Model saved with accuracy: 44.58%
Epoch 2/2000, Loss: 0.6732, Test Accuracy: 46.15%, Best accuracy: 44.58%, Epoch best accuracy: 1, seed: 79
✅ Model saved with accuracy: 46.15%
Epoch 3/2000, Loss: 0.6722, Test Accuracy: 47.34%, Best accuracy: 46.15%, Epoch best accuracy: 2, seed: 79
✅ Model saved with accuracy: 47.34%
Epoch 4/2000, Loss: 0.6719, Test Accuracy: 47.73%, Best accuracy: 47.34%, Epoch best accuracy: 3, seed: 79
✅ Model saved with accuracy: 47.73%
Epoch 5/2000, Loss: 0.6715, Test Accuracy: 48.92%, Best accuracy: 47.73%, Epoch best accuracy: 4, seed: 79
✅ Model saved with accuracy: 48.92%
Epoch 6/2000, Loss: 0.6706, Test Accuracy: 47.34%, Best accuracy: 48.92%, Epoch best accuracy: 5, seed: 79
Epoch 7/2000, Loss: 0.6702, Test Accuracy: 48.92%, Best accuracy: 48.92%, Epoch best accuracy: 5, seed: 79
Epoch 8/2000, Loss: 0.6696, Test Accuracy: 48.13%, Best accuracy: 48.92%

🔹 Как использовать модель для предсказания?  
После загрузки модели можно предсказать направление следующей свечи:

In [88]:
# === Предсказание следующей свечи ===
# Загружаем модель
model.load_state_dict(torch.load("best_model_first_MIX_01.pth"))
model.eval()

# Берем последние 20 свечей из df_fut
last_sequence = torch.tensor(
    df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long
).unsqueeze(0).to(device)

print(last_sequence)

# Предсказание
with torch.no_grad():
    logit = model(last_sequence).item()  # Логит
    probability_up = torch.sigmoid(torch.tensor(logit)).item()  # Применяем сигмоиду
    probability_down = 1 - probability_up  # Вероятность падения

    direction = "📈 UP" if probability_up >= 0.5 else "📉 DOWN"

    print(f"Prediction: {direction}")
    print(
        f"UP Probability: {probability_up:.2%}, \n"
        f"DOWN Probability: {probability_down:.2%}"
    )

tensor([[12,  8, 12,  8, 17, 13,  8,  9,  6, 17, 17,  8, 10, 17,  8,  2,  5,  7,
         10, 17]])
Prediction: 📈 UP
UP Probability: 56.57%, 
DOWN Probability: 43.43%


In [89]:
df_fut

,TRADEDATE,OPEN,LOW,HIGH,CLOSE,VOLUME,CANDLE_CODE,CANDLE_INT
0,2015-01-05,142000.0,138525.0,145950.0,145150.0,2587,112,14
1,2015-01-06,144750.0,144350.0,149900.0,149900.0,2953,100,9
2,2015-01-08,149500.0,148200.0,158200.0,156025.0,5567,111,13
3,2015-01-09,155575.0,150450.0,158100.0,154975.0,6100,022,8
4,2015-01-12,154750.0,151975.0,155500.0,152850.0,4574,011,4
...,...,...,...,...,...,...,...,...
2548,2025-02-27,332100.0,324325.0,332100.0,328325.0,271316,002,2
2549,2025-02-28,328000.0,319875.0,328825.0,324900.0,243498,012,5
2550,2025-03-03,324775.0,316550.0,330300.0,319050.0,334920,021,7
2551,2025-03-04,318850.0,316700.0,331400.0,331050.0,246511,101,10


In [90]:
# === ЗАГРУЗКА ОБУЧЕННОЙ МОДЕЛИ ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = "best_model_first_MIX_01.pth"
model = CandleLSTM(
    vocab_size=len(unique_codes), embedding_dim=8, hidden_dim=32, output_dim=1
    ).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# === ПРОГНОЗИРОВАНИЕ ===
predictions = []
with torch.no_grad():
    for i in range(len(df_fut) - window_size):
        sequence = torch.tensor(
            df_fut['CANDLE_INT'].iloc[i:i+window_size].values, dtype=torch.long
            ).unsqueeze(0).to(device)
        pred = model(sequence).item()
        predictions.append(1 if pred > 0.5 else 0)

# Заполняем колонку PREDICTION (первые window_size значений - NaN)
df_fut['PREDICTION'] = [None] * window_size + predictions

# === СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ===
predictions_file = r"predictions_first_model_01.csv"
df_fut.to_csv(predictions_file, index=False)
print(f"✅ Прогнозы сохранены в '{predictions_file}'")

df_fut

✅ Прогнозы сохранены в 'predictions_first_model_01.csv'


,TRADEDATE,OPEN,LOW,HIGH,CLOSE,VOLUME,CANDLE_CODE,CANDLE_INT,PREDICTION
0,2015-01-05,142000.0,138525.0,145950.0,145150.0,2587,112,14,NaN
1,2015-01-06,144750.0,144350.0,149900.0,149900.0,2953,100,9,NaN
2,2015-01-08,149500.0,148200.0,158200.0,156025.0,5567,111,13,NaN
3,2015-01-09,155575.0,150450.0,158100.0,154975.0,6100,022,8,NaN
4,2015-01-12,154750.0,151975.0,155500.0,152850.0,4574,011,4,NaN
...,...,...,...,...,...,...,...,...,...
2548,2025-02-27,332100.0,324325.0,332100.0,328325.0,271316,002,2,0.0
2549,2025-02-28,328000.0,319875.0,328825.0,324900.0,243498,012,5,1.0
2550,2025-03-03,324775.0,316550.0,330300.0,319050.0,334920,021,7,1.0
2551,2025-03-04,318850.0,316700.0,331400.0,331050.0,246511,101,10,1.0


# Глобальная проверка.  
2025-02-18,338500.0,331675.0,342525.0,332250.0,386951,020,6,0.0  
2025-02-19,332225.0,327850.0,335650.0,334500.0,216330,122,17,1.0  
2025-02-20,334750.0,333625.0,339600.0,335375.0,179755,122,17,1.0  
2025-02-21,335675.0,331325.0,337600.0,334225.0,164931,022,8,0.0  

Есть код:  
```Python
db_path = Path(r'C:\Users\Alkor\gd\data_quote_db\MIX_futures_day.db')  
  
with sqlite3.connect(db_path) as conn:  
    df_fut = pd.read_sql_query(  
        "SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME FROM Day",  
        conn  
    )  
```
Как в дата фрейм выбрать все строки до даты 2025-02-18 в колонке TRADEDATE

In [91]:
# === Делаем предсказание на 2025-02-18 ===
# === Загрузка данных до 2025-02-18 ===
with sqlite3.connect(db_path) as conn:
    df_fut = pd.read_sql_query(
        """
        SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME 
        FROM Day 
        WHERE TRADEDATE < '2025-02-18'
        ORDER BY TRADEDATE
        """,
        conn
    )

# === Функция кодирования свечей ===
def encode_candle(row):
    open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']
    direction = 1 if close > open_ else (0 if close < open_ else 2)
    upper_shadow = high - max(open_, close)
    lower_shadow = min(open_, close) - low
    body = abs(close - open_)
    
    def classify_shadow(shadow, body):
        return 0 if shadow < 0.1 * body else (1 if shadow < 0.5 * body else 2)

    return f"{direction}{classify_shadow(upper_shadow, body)}{classify_shadow(lower_shadow, body)}"

df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# === Преобразование кодов в числа ===
unique_codes = sorted(df_fut['CANDLE_CODE'].unique())
code_to_int = {code: i for i, code in enumerate(unique_codes)}
df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

# === Предсказание следующей свечи ===
# Загружаем модель
model.load_state_dict(torch.load("best_model_first_MIX_01.pth"))
model.eval()

# Берем последние 20 свечей из df_fut
last_sequence = torch.tensor(
    df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long
).unsqueeze(0).to(device)

# Предсказание
with torch.no_grad():
    logit = model(last_sequence).item()  # Логит
    print(logit)
    probability_up = torch.sigmoid(torch.tensor(logit)).item()  # Применяем сигмоиду
    probability_down = 1 - probability_up  # Вероятность падения

    direction = "📈 UP" if probability_up >= 0.5 else "📉 DOWN"

    print(f"Prediction: {direction}")
    print(
        f"UP Probability: {probability_up:.2%}, \n"
        f"DOWN Probability: {probability_down:.2%}"
    )

-0.9402869939804077
Prediction: 📉 DOWN
UP Probability: 28.08%, 
DOWN Probability: 71.92%


Предсказание на 2025-02-18 на понижение, что `соответствует` и колонке предсказаний в csv файле в строке на 2025-02-18.

In [92]:
# === Делаем предсказание на 2025-02-19 ===
# === Загрузка данных до 2025-02-19 ===
with sqlite3.connect(db_path) as conn:
    df_fut = pd.read_sql_query(
        """
        SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME 
        FROM Day 
        WHERE TRADEDATE < '2025-02-19'
        ORDER BY TRADEDATE
        """,
        conn
    )

# === Функция кодирования свечей ===
def encode_candle(row):
    open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']
    direction = 1 if close > open_ else (0 if close < open_ else 2)
    upper_shadow = high - max(open_, close)
    lower_shadow = min(open_, close) - low
    body = abs(close - open_)
    
    def classify_shadow(shadow, body):
        return 0 if shadow < 0.1 * body else (1 if shadow < 0.5 * body else 2)

    return f"{direction}{classify_shadow(upper_shadow, body)}{classify_shadow(lower_shadow, body)}"

df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# === Преобразование кодов в числа ===
unique_codes = sorted(df_fut['CANDLE_CODE'].unique())
code_to_int = {code: i for i, code in enumerate(unique_codes)}
df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

# === Предсказание следующей свечи ===
# Загружаем модель
model.load_state_dict(torch.load("best_model_first_MIX_01.pth"))
model.eval()

# Берем последние 20 свечей из df_fut
last_sequence = torch.tensor(
    df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long
).unsqueeze(0).to(device)

# Предсказание
with torch.no_grad():
    logit = model(last_sequence).item()  # Логит
    print(logit)
    probability_up = torch.sigmoid(torch.tensor(logit)).item()  # Применяем сигмоиду
    probability_down = 1 - probability_up  # Вероятность падения

    direction = "📈 UP" if probability_up >= 0.5 else "📉 DOWN"

    print(f"Prediction: {direction}")
    print(
        f"UP Probability: {probability_up:.2%}, \n"
        f"DOWN Probability: {probability_down:.2%}"
    )

0.691784143447876
Prediction: 📈 UP
UP Probability: 66.64%, 
DOWN Probability: 33.36%


Предсказание на 2025-02-19 на повышение, что `соответствует` и колонке предсказаний в csv файле в строке на 2025-02-19.

In [93]:
# === Делаем предсказание на 2025-02-20 ===
# === Загрузка данных до 2025-02-20 ===
with sqlite3.connect(db_path) as conn:
    df_fut = pd.read_sql_query(
        """
        SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME 
        FROM Day 
        WHERE TRADEDATE < '2025-02-20'
        ORDER BY TRADEDATE
        """,
        conn
    )

# === Функция кодирования свечей ===
def encode_candle(row):
    open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']
    direction = 1 if close > open_ else (0 if close < open_ else 2)
    upper_shadow = high - max(open_, close)
    lower_shadow = min(open_, close) - low
    body = abs(close - open_)
    
    def classify_shadow(shadow, body):
        return 0 if shadow < 0.1 * body else (1 if shadow < 0.5 * body else 2)

    return f"{direction}{classify_shadow(upper_shadow, body)}{classify_shadow(lower_shadow, body)}"

df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# === Преобразование кодов в числа ===
unique_codes = sorted(df_fut['CANDLE_CODE'].unique())
code_to_int = {code: i for i, code in enumerate(unique_codes)}
df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

# === Предсказание следующей свечи ===
# Загружаем модель
model.load_state_dict(torch.load("best_model_first_MIX_01.pth"))
model.eval()

# Берем последние 20 свечей из df_fut
last_sequence = torch.tensor(
    df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long
).unsqueeze(0).to(device)

# Предсказание
with torch.no_grad():
    logit = model(last_sequence).item()  # Логит
    print(logit)
    probability_up = torch.sigmoid(torch.tensor(logit)).item()  # Применяем сигмоиду
    probability_down = 1 - probability_up  # Вероятность падения

    direction = "📈 UP" if probability_up >= 0.5 else "📉 DOWN"

    print(f"Prediction: {direction}")
    print(
        f"UP Probability: {probability_up:.2%}, \n"
        f"DOWN Probability: {probability_down:.2%}"
    )

1.18339204788208
Prediction: 📈 UP
UP Probability: 76.56%, 
DOWN Probability: 23.44%


Предсказание на 2025-02-20 на повышение, что `соответствует` и колонке предсказаний в csv файле в строке на 2025-02-20.

In [94]:
# === Делаем предсказание на 2025-02-21 ===
# === Загрузка данных до 2025-02-21 ===
with sqlite3.connect(db_path) as conn:
    df_fut = pd.read_sql_query(
        """
        SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME 
        FROM Day 
        WHERE TRADEDATE < '2025-02-21'
        ORDER BY TRADEDATE
        """,
        conn
    )

# === Функция кодирования свечей ===
def encode_candle(row):
    open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']
    direction = 1 if close > open_ else (0 if close < open_ else 2)
    upper_shadow = high - max(open_, close)
    lower_shadow = min(open_, close) - low
    body = abs(close - open_)
    
    def classify_shadow(shadow, body):
        return 0 if shadow < 0.1 * body else (1 if shadow < 0.5 * body else 2)

    return f"{direction}{classify_shadow(upper_shadow, body)}{classify_shadow(lower_shadow, body)}"

df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# === Преобразование кодов в числа ===
unique_codes = sorted(df_fut['CANDLE_CODE'].unique())
code_to_int = {code: i for i, code in enumerate(unique_codes)}
df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

# === Предсказание следующей свечи ===
# Загружаем модель
model.load_state_dict(torch.load("best_model_first_MIX_01.pth"))
model.eval()

# Берем последние 20 свечей из df_fut
last_sequence = torch.tensor(
    df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long
).unsqueeze(0).to(device)

# Предсказание
with torch.no_grad():
    logit = model(last_sequence).item()  # Логит
    print(logit)
    probability_up = torch.sigmoid(torch.tensor(logit)).item()  # Применяем сигмоиду
    probability_down = 1 - probability_up  # Вероятность падения

    direction = "📈 UP" if probability_up >= 0.5 else "📉 DOWN"

    print(f"Prediction: {direction}")
    print(
        f"UP Probability: {probability_up:.2%}, \n"
        f"DOWN Probability: {probability_down:.2%}"
    )

0.4870644509792328
Prediction: 📈 UP
UP Probability: 61.94%, 
DOWN Probability: 38.06%


Предсказание на 2025-02-21 на повышение, что `не соответствует` колонке предсказаний в csv файле в строке на 2025-02-21. Там значение 0.0.

In [95]:
# === Делаем предсказание на 2025-02-22(24) ===
# === Загрузка данных до 2025-02-22 ===
with sqlite3.connect(db_path) as conn:
    df_fut = pd.read_sql_query(
        """
        SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME 
        FROM Day 
        WHERE TRADEDATE < '2025-02-22'
        ORDER BY TRADEDATE
        """,
        conn
    )

# === Функция кодирования свечей ===
def encode_candle(row):
    open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']
    direction = 1 if close > open_ else (0 if close < open_ else 2)
    upper_shadow = high - max(open_, close)
    lower_shadow = min(open_, close) - low
    body = abs(close - open_)
    
    def classify_shadow(shadow, body):
        return 0 if shadow < 0.1 * body else (1 if shadow < 0.5 * body else 2)

    return f"{direction}{classify_shadow(upper_shadow, body)}{classify_shadow(lower_shadow, body)}"

df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# === Преобразование кодов в числа ===
unique_codes = sorted(df_fut['CANDLE_CODE'].unique())
code_to_int = {code: i for i, code in enumerate(unique_codes)}
df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

# === Предсказание следующей свечи ===
# Загружаем модель
model.load_state_dict(torch.load("best_model_first_MIX_01.pth"))
model.eval()

# Берем последние 20 свечей из df_fut
last_sequence = torch.tensor(
    df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long
).unsqueeze(0).to(device)

# Предсказание
with torch.no_grad():
    logit = model(last_sequence).item()  # Логит
    print(logit)
    probability_up = torch.sigmoid(torch.tensor(logit)).item()  # Применяем сигмоиду
    probability_down = 1 - probability_up  # Вероятность падения

    direction = "📈 UP" if probability_up >= 0.5 else "📉 DOWN"

    print(f"Prediction: {direction}")
    print(
        f"UP Probability: {probability_up:.2%}, \n"
        f"DOWN Probability: {probability_down:.2%}"
    )

0.305134654045105
Prediction: 📈 UP
UP Probability: 57.57%, 
DOWN Probability: 42.43%
